<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Tracing CrewAI with Arize Phoenix - Orchestrator-Workers Workflow</h1>

In [ ]:
%pip install -q arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp crewai crewai_tools openinference-instrumentation-crewai

# Set up Keys and Dependencies

Note: For this colab you'll need:

*   OpenAI API key (https://openai.com/)
*   Serper API key (https://serper.dev/)
*   Phoenix API key (https://app.phoenix.arize.com/)

In [ ]:
import getpass
import os

# Prompt the user for their API keys if they haven't been set
openai_key = os.getenv("OPENAI_API_KEY", "OPENAI_API_KEY")
serper_key = os.getenv("SERPER_API_KEY", "SERPER_API_KEY")

if openai_key == "OPENAI_API_KEY":
    openai_key = getpass.getpass("Please enter your OPENAI_API_KEY: ")

if serper_key == "SERPER_API_KEY":
    serper_key = getpass.getpass("Please enter your SERPER_API_KEY: ")

# Set the environment variables with the provided keys
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["SERPER_API_KEY"] = serper_key

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass.getpass("Enter your Phoenix API key: ")

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

## Configure Tracing

In [ ]:
from phoenix.otel import register

tracer_provider = register(
    project_name="crewai-agents", endpoint="https://app.phoenix.arize.com/v1/traces"
)

# Instrument CrewAI

In [ ]:
from openinference.instrumentation.crewai import CrewAIInstrumentor

CrewAIInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

## Define your Agents

In [ ]:
from crewai import Agent, Crew, Task

# Define worker agents
trend_researcher = Agent(
    role="AI Trend Researcher",
    goal="Analyze current advancements in AI",
    backstory="Expert in tracking and analyzing new trends in artificial intelligence.",
    verbose=True,
)

policy_analyst = Agent(
    role="AI Policy Analyst",
    goal="Examine the implications of AI regulations and governance",
    backstory="Tracks AI policy developments across governments and organizations.",
    verbose=True,
)

risk_specialist = Agent(
    role="AI Risk Specialist",
    goal="Identify potential risks in frontier AI development",
    backstory="Focuses on safety, alignment, and misuse risks related to advanced AI.",
    verbose=True,
)

synthesizer = Agent(
    role="Synthesis Writer",
    goal="Summarize all findings into a final cohesive report",
    backstory="Expert at compiling research insights into executive-level narratives.",
    verbose=True,
)

orchestrator = Agent(
    role="Orchestrator",
    goal=(
        "Your job is to delegate research and writing tasks to the correct coworker using the 'Delegate work to coworker' tool.\n"
        "For each task you assign, you MUST call the tool with the following JSON input:\n\n"
        "{\n"
        '  "task": "Short summary of the task to do (plain string)",\n'
        '  "context": "Why this task is important or part of the report (plain string)",\n'
        '  "coworker": "One of: AI Trend Researcher, AI Policy Analyst, AI Risk Specialist, Synthesis Writer"\n'
        "}\n\n"
        "IMPORTANT:\n"
        "- Do NOT format 'task' or 'context' as dictionaries.\n"
        "- Do NOT include types or nested descriptions.\n"
        "- Only use plain strings for both.\n"
        "- Call the tool multiple times, one per coworker."
    ),
    backstory="You are responsible for assigning each part of an AI report to the right specialist.",
    verbose=True,
    allow_delegation=True,
)

## Define your Tasks

In [ ]:
# Define the initial task only for the orchestrator
initial_task = Task(
    description="Create an AI trends report. It should include recent innovations, policy updates, and safety risks. Then synthesize it into a unified summary.",
    expected_output="Assign subtasks via the DelegateWorkTool and return a final report.",
    agent=orchestrator,
)

## Create Crew

In [ ]:
crew = Crew(
    agents=[trend_researcher, policy_analyst, risk_specialist, synthesizer],
    tasks=[initial_task],
    manager_agent=orchestrator,
    verbose=True,
)
# Run the full workflow
result = crew.kickoff()
print(result)

### Check your Phoenix project to view the traces and spans from your runs.